In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
def linear_layer(X, n_input, n_output, activation=False, scope=False):
    
    with tf.variable_scope(scope or 'Linear', reuse=tf.AUTO_REUSE):
        
        W = tf.get_variable(name='W',
                           shape=[n_input, n_output],
                           initializer=tf.random_normal_initializer(mean=0.0, stddev=0.1))
        b = tf.get_variable(name='b',
                           shape=[n_output],
                           initializer=tf.constant_initializer())
        h = tf.matmul(X,W) + b
        
        if activation is not None:
            h = activation(h)
            
        return h

In [ ]:
def distance(a, b):
    return tf.abs(a-b)

In [ ]:
n_classes = 10
n_samples = mnist.train.num_examples
X = tf.placeholder(dtype='float', shape=[None, 784], name='X')
y = tf.placeholder(dtype='float', shape=[None, n_classes], name='y')

In [ ]:
network = [784, 256, 10]
current_input = X

for i in range(1,len(network)):
    current_input = linear_layer(current_input,
                                 n_input= network[i-1], 
                                 n_output= network[i],
                                 activation= tf.nn.softmax if (i+1)<len(network) else None,
                                 scope= 'layer_' + str(i))


y_pred = current_input

In [ ]:
learning_rate = 0.5
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [ ]:
training_epochs = 500
batch_size = 100
    
sess.run(tf.initialize_all_variables())

for i in range(training_epochs):
    n_batches = n_samples//batch_size
    for n in range(n_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={X:batch_x, y:batch_y})
    #print('Epoch {}: done.'.format(i+1))

print('Training done.')
    
    

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={X: mnist.test.images, y: mnist.test.labels}))